In [11]:
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = 0 # for dice special

# Libraries
from sklearn.model_selection import KFold
import numpy as np
import argparse
import yaml

# accelerate
from accelerate.utils import ProjectConfiguration, set_seed
from accelerate import Accelerator
from accelerate.logging import get_logger

import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.nn.modules.loss import CrossEntropyLoss
import logging
from tqdm import tqdm
from importlib import import_module
import wandb
from sklearn.metrics import jaccard_score

from monai.transforms import (
    Compose,
    RandFlipd,
    RandRotated,
    RandZoomd,
    ScaleIntensityd,
    EnsureTyped,
    EnsureChannelFirstd,
    Resized,
    RandGaussianNoised,
    RandGaussianSmoothd,
    Rand2DElasticd,
    RandAffined,
    OneOf,
)

# extra imports
from datasets_utils.datasets import ABUS_dataset
sys.path.append(str(repo_path / 'SAMed'))
from SAMed.SAMed_h.segment_anything import sam_model_registry
from SAMed.utils import DiceLoss #, Focal_loss

In [12]:
# get SAM model
sam, _ = sam_model_registry['vit_h'](image_size=512,
                                    num_classes=2,
                                    checkpoint=str(repo_path /'checkpoints/sam_vit_h_4b8939.pth'),
                                    pixel_mean=[0, 0, 0],
                                    pixel_std=[1, 1, 1])
# # load lora model
# pkg = import_module('sam_lora_image_encoder')
# net = pkg.LoRA_Sam(sam, 4) # lora rank is 4